# Experiment: Train Tensorflow DistilBERT Model Experiment

## Confirm Environment

In [1]:
!conda info


     active environment : base
    active env location : /shared/EL9/explorer/anaconda3/2024.06
            shell level : 1
       user config file : /home/neiderer.c/.condarc
 populated config files : 
          conda version : 24.5.0
    conda-build version : 24.5.1
         python version : 3.12.4.final.0
                 solver : libmamba (default)
       virtual packages : __archspec=1=broadwell
                          __conda=24.5.0=0
                          __cuda=12.3=0
                          __glibc=2.34=0
                          __linux=5.14.0=0
                          __unix=0=0
       base environment : /shared/EL9/explorer/anaconda3/2024.06  (read only)
      conda av data dir : /shared/EL9/explorer/anaconda3/2024.06/etc/conda
  conda av metadata url : None
           channel URLs : https://repo.anaconda.com/pkgs/main/linux-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/r/linux-64

## Setup and Imports

In [2]:
from emolex.preprocessing import load_mental_health_sentiment_dataset, clean_text, encode_sentiment_labels, split_data, dl_text_vectorization
from emolex.train import train_distilbert_model_tf
from emolex.evaluation import plot_training_history, generate_confusion_matrix, generate_classification_report
from emolex.utils import detect_and_set_device

2025-07-03 00:23:41.996349: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751516622.016768 4145173 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751516622.023146 4145173 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751516622.039645 4145173 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751516622.039659 4145173 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751516622.039661 4145173 computation_placer.cc:177] computation placer alr

## Device Setup

In [3]:
# Detect and set up GPU or use CPU
device_used = detect_and_set_device()
print(f"TensorFlow is configured to use: {device_used}")

GPU is available. Attempting to use GPU.
Successfully configured GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow is configured to use: GPU


## Load Data

In [4]:
df = load_mental_health_sentiment_dataset()
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51093 entries, 0 to 51092
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    51093 non-null  object
 1   label   51093 non-null  object
dtypes: object(2)
memory usage: 798.5+ KB


,text,label
0,oh my gosh,Anxiety
1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety
3,I've shifted my focus to something else but I'...,Anxiety
4,"I'm restless and restless, it's been a month n...",Anxiety


## Clean Data

In [5]:
print(f"\n--- Cleaning Text ---")
df['clean_text'] = df["text"].apply(clean_text)
print("Text cleaning complete. Sample cleaned text:")
print("\n", df[["text", "clean_text"]].sample(5))


--- Cleaning Text ---
Text cleaning complete. Sample cleaned text:

                                                     text  \
35113  Fear of Neurological disease because I suck at...   
11113  Sorry about the title, just wanted to grab att...   
47438  Genuinely, why live? I don’t enjoy being alive...   
37712  i feel the need of depending on people for me ...   
6400   Girls who are often hurt will be surprised if ...   

                                              clean_text  
35113  fear neurological disease suck balancing one l...  
11113  sorry title wanted grab attention slowly recov...  
47438  genuinely live dont enjoy alive dont understan...  
37712  feel need depending people feel better comfort...  
6400   girl often hurt surprised meet good patient ca...  


## Encode Labels

In [6]:
print(f"\n--- Encoding Labels ---")
df, encoder = encode_sentiment_labels(df)
print("Label encoding complete. Sample encoded labels:")
print("\n", df[['label', 'label_encoded']].sample(5))


--- Encoding Labels ---
Label Encoding Map: {'Anxiety': 0, 'Bipolar': 1, 'Depression': 2, 'Normal': 3, 'Personality disorder': 4, 'Stress': 5, 'Suicidal': 6}
Label encoding complete. Sample encoded labels:

             label  label_encoded
20358  Depression              2
5912       Normal              3
16267  Depression              2
39434  Depression              2
32777      Normal              3


## Train-Test Split

In [7]:
print("\n--- Perform Train-Test Split ---")
X_train_raw, X_test_raw, y_train, y_test = split_data(df) 
print(f"Train set size: {len(X_train_raw)} samples")
print(f"Test set size: {len(X_test_raw)} samples")


--- Perform Train-Test Split ---
Train set size: 40874 samples
Test set size: 10219 samples


## Train Model

In [9]:
model, history = train_distilbert_model_tf(X_train_raw, y_train, X_test_raw, y_test, len(encoder.classes_), epochs=3)

Initializing DistilBERT tokenizer...
Creating Hugging Face Datasets from input data...


/home/neiderer.c/.local/lib/python3.9/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Tokenizing datasets...


Map:   0%|          | 0/40874 [00:00<?, ? examples/s]

Map:   0%|          | 0/10219 [00:00<?, ? examples/s]

...

Loading TFDistilBertForSequenceClassification model with 7 labels...


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Compiling model...
Starting DistilBERT model training for 3 epochs...
Epoch 1/3


I0000 00:00:1751516688.461976 4145261 service.cc:152] XLA service 0x150744644580 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751516688.462004 4145261 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-12GB, Compute Capability 6.0
2025-07-03 00:24:48.469645: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
E0000 00:00:1751516688.489562 4145261 cuda_dnn.cc:522] Loaded runtime CuDNN library: 9.1.0 but source was compiled with: 9.3.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
E0000 00:00:1751516688.492653 4145261 cuda_dnn.cc:522] Loaded runtime CuDNN library: 9.1.0 but source was comp

FailedPreconditionError: Graph execution error:

Detected at node Adam/StatefulPartitionedCall_103 defined at (most recent call last):
  File "/home/neiderer.c/.conda/envs/tensorflow-gpu-neiderer/lib/python3.9/runpy.py", line 197, in _run_module_as_main

  File "/home/neiderer.c/.conda/envs/tensorflow-gpu-neiderer/lib/python3.9/runpy.py", line 87, in _run_code

  File "/home/neiderer.c/.local/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/neiderer.c/.local/lib/python3.9/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/neiderer.c/.local/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/neiderer.c/.local/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 211, in start

  File "/home/neiderer.c/.conda/envs/tensorflow-gpu-neiderer/lib/python3.9/asyncio/base_events.py", line 601, in run_forever

  File "/home/neiderer.c/.conda/envs/tensorflow-gpu-neiderer/lib

## Evaluate Model

In [ ]:
print("\n--- Plot Training History ---")
plot_training_history(history)

In [ ]:
print("\n--- Predict Test Classes ---")
y_pred = model.predict(X_test_pad_filtered)
y_pred_classes = y_pred.argmax(axis=1)

In [ ]:
print("\n--- Generate Confusion Matrix ---")
fig, ax = generate_confusion_matrix(y_test_filtered, y_pred_classes, class_labels=encoder.classes_)

In [ ]:
print("\n--- Generate Classification Report ---")
generate_classification_report(y_test_filtered, y_pred_classes, class_labels=encoder.classes_)